In [ ]:
import matplotlib.pyplot as plt
import matplotlib
import tensorflow as tf
import cv2
from sklearn.model_selection import train_test_split
import os
import numpy as np

In [ ]:
from PIL import Image, ImageEnhance
import os

def increase_brightness(input_path, output_path, brightness_factor):
    # Open the image file
    with Image.open(input_path) as img:
        # Enhance the brightness
        enhancer = ImageEnhance.Brightness(img)
        brightened_img = enhancer.enhance(brightness_factor)

        # Save the brightened image
        brightened_img.save(output_path)


In [ ]:
for i in images:
    input_image_path = "massachusetts-buildings-dataset/png/train/"+i

    # Specify the output directory for the brightened image
    output_directory = "Sample"

    # Ensure the output directory exists
    os.makedirs(output_directory, exist_ok=True)

    # Specify the brightness factor (1.0 is unchanged, values greater than 1 increase brightness)
    brightness_factor = 1.5

    # Generate the output file path by combining the output directory and input image filename
    input_filename = os.path.basename(input_image_path)
    output_image_path = os.path.join(output_directory, f"{input_filename}")

    # Increase brightness and save the image
    increase_brightness(input_image_path, output_image_path, brightness_factor)

In [ ]:
import os
import cv2
import numpy as np
from skimage import io

# Specify the paths to the original images and masks
original_images_path = 'brightned'
mask_images_path = 'massachusetts-buildings-dataset/png/train_labels'

# Specify the output directory for the augmented images and masks
output_images_path = 'train_1'
output_masks_path = 'masks_1'

# Specify the new output location
new_output_images_path = 'train_images'
new_output_masks_path = 'train_masks'

# Create output directories if they don't exist
os.makedirs(new_output_images_path, exist_ok=True)
os.makedirs(new_output_masks_path, exist_ok=True)

# Function to apply random transformations
def apply_random_transformations(image, mask):
    # Randomly choose whether to apply transformations
    if np.random.rand() > 0.5:
        # Apply Gaussian filter
        sigma = np.random.uniform(0.5, 2.0)
        image = cv2.GaussianBlur(image, (0, 0), sigma)

    if np.random.rand() > 0.5:
        # Apply 90-degree rotation
        image = np.rot90(image)
        mask = np.rot90(mask)

    return image, mask

# Desired number of augmented images
target_num_images = 250

# Counter for the number of augmented images created
num_created_images = 0

# Iterate until the desired number of images is reached
while num_created_images < target_num_images:
    # Iterate over each original image
    for filename in os.listdir(original_images_path):
        if filename.endswith('.png'):  # Adjust the extension based on your image format
            # Load original image and corresponding mask
            image_path = os.path.join(original_images_path, filename)
            mask_path = os.path.join(mask_images_path, filename)

            original_image = io.imread(image_path)
            mask_image = io.imread(mask_path)

            # Apply random transformations
            augmented_image, augmented_mask = apply_random_transformations(original_image, mask_image)

            # Save augmented images and masks to the new output location
            new_output_image_path = os.path.join(new_output_images_path, f"{num_created_images}_{filename}")
            new_output_mask_path = os.path.join(new_output_masks_path, f"{num_created_images}_{filename}")

            io.imsave(new_output_image_path, augmented_image)
            io.imsave(new_output_mask_path, augmented_mask)

            num_created_images += 1

            # Break the loop if the desired number of images is reached
            if num_created_images == target_num_images:
                break

# Now, you have generated and saved 250 augmented images and masks in the specified directories.


In [ ]:
base_dir = 'massachusetts-buildings-dataset/png'
images_dir ='train_images'
masks_dir = 'train_masks'
val_images_dir ='massachusetts-buildings-dataset/png/val'
val_masks_dir = 'massachusetts-buildings-dataset/png/val_labels'
test_images_dir='massachusetts-buildings-dataset/png/test'
test_masks_dir='massachusetts-buildings-dataset/png/test_labels'
images_listdir = os.listdir(images_dir)
masks_listdir = os.listdir(masks_dir)
random_images = np.random.choice(masks_listdir, size = 9, replace = False)
test_images_listdir = os.listdir(test_images_dir)
test_masks_listdir = os.listdir(test_masks_dir)
image_size= 224
input_image_size=(1500,1500)
def read_image(path):
    img = cv2.imread(path)
    img = cv2.resize(img, (image_size, image_size))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img
number= 250
rows = 3
cols = 3
fig, ax = plt.subplots(rows, cols, figsize = (10,10))
for i, ax in enumerate(ax.flat):
    if i < len(random_images):
        img = read_image(f"{images_dir}/{random_images[i]}")
        ax.set_title(f"{random_images[i]}")
        ax.imshow(img)
        ax.axis('off')
fig, ax = plt.subplots(rows, cols, figsize = (10,10))
for i, ax in enumerate(ax.flat):
    if i < len(random_images):
        file=random_images[i]
        if os.path.exists(os.path.join(masks_dir,file)):
            img = read_image(f"{masks_dir}/{file}")
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            ax.set_title(f"{random_images[i]}")
            ax.imshow(img)
            ax.axis('off')
        else:
            print('not exist')

MASKS=np.zeros((1,image_size, image_size, 1), dtype=bool)
IMAGES=np.zeros((1,image_size, image_size, 3),dtype=np.uint8)

for j,file in enumerate(masks_listdir[0:number]):   ##the smaller, the faster
    try:
        image = read_image(f"{images_dir}/{file}")
        image_ex = np.expand_dims(image, axis=0)
        IMAGES = np.vstack([IMAGES, image_ex])
        file2=file[0:-4]+'.png'
        mask = read_image(f"{masks_dir}/{file2}")
        mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
        mask = mask.reshape(image_size,image_size,1)
        mask_ex = np.expand_dims(mask, axis=0)
        MASKS = np.vstack([MASKS, mask_ex])
    except:
        print(file)
        continue
TMASKS=np.zeros((1,image_size, image_size,1), dtype=bool)
TIMAGES=np.zeros((1,image_size, image_size,3),dtype=np.uint8)

for j,file in enumerate(test_images_listdir): ##the smaller, the faster
    try:
        image = read_image(f"{test_images_dir}/{file}")
        image_ex = np.expand_dims(image, axis=0)
        TIMAGES = np.vstack([TIMAGES, image_ex])
        file2=file[0:-4]+'.png'
        mask = read_image(f"{test_masks_dir}/{file2}")
        mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
        mask = mask.reshape(image_size,image_size,1)
        mask_ex = np.expand_dims(mask, axis=0)
        TMASKS = np.vstack([TMASKS, mask_ex])
    except:
        print(file)
        continue
images=np.array(IMAGES)[1:number+1]
masks=np.array(MASKS)[1:number+1]
test_images=np.array(TIMAGES)[1:]
test_masks=np.array(TMASKS[1:])
images_train, images_test, masks_train, masks_test = train_test_split(images, masks, test_size=0.2, random_state=43)

print(images_train.shape)
print(images_test.shape)
images_train = images_train/255.0
images_val = images_test/255.0
masks_train =  masks_train/255.0
masks_val = masks_test/255.0

In [ ]:
def squeeze_excite_block(tensor,ratio=16):
    init = tensor
    channel_axis = 1 if K.image_data_format() == "channels_first" else -1
    filters = init.shape[channel_axis]
    se_shape = (1, 1, filters)

    se = GlobalAveragePooling2D()(init)
    se = Reshape(se_shape)(se)
    se = Dense(filters // ratio, activation='relu', kernel_initializer='he_normal', use_bias=False)(se)
    se = Dense(filters, activation='relu', kernel_initializer='he_normal', use_bias=True)(se)
    se = Dense(filters, activation='relu', kernel_initializer='he_normal', use_bias=True)(se)
    if K.image_data_format() == 'channels_first':
        se = Permute((3, 1, 2))(se)
    x = multiply([init, se],name='squeeze_and_excitation')
    return x

In [ ]:
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input,Dropout,GlobalAveragePooling2D,Reshape,Dense,Permute,multiply
from tensorflow.keras.models import Model
from tensorflow.keras.applications import ResNet50
import tensorflow.keras.backend as K

def conv_block(input, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(input)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    return x

def decoder_block(input, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(input)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

def build_resnet50_unet(input_shape):

    inputs = Input(input_shape)


    resnet50 = ResNet50(include_top=False,input_tensor=inputs)
    resnet50.trainable = True


    s1 = resnet50.get_layer("input_15").output  ## (224 x 224)
    s2 = resnet50.get_layer("conv1_relu").output        ## (112 x 112)
    s3 = resnet50.get_layer("conv2_block3_out").output  ## (56 x 56)
    s4 = resnet50.get_layer("conv3_block4_out").output  ## (28 x 28)


    b1 = resnet50.get_layer("conv4_block6_out").output  ## (14 x 14)

    d1 = decoder_block(squeeze_excite_block(b1), s4, 512) #(28 x 28)
    d2 = decoder_block(d1, s3, 256)                     ## (56 x 56)
    d3 = decoder_block(d2, s2, 128)                     ## (112 x 112)
    d4 = decoder_block(d3, s1, 64)                      ##(224 x 224)

    """ Output """
    outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(d4)

    model = Model(inputs, outputs, name="ResNet50_U-Net-XE")

    return model,b1

if __name__ == "__main__":
    input_shape = (224, 224, 3)
    model,b1 = build_resnet50_unet(input_shape)


In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
learning_rate = 0.001  # Adjust the value based on your specific task and observations

# Initialize the Adam optimizer with the specified learning rate
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

callback = tf.keras.callbacks.EarlyStopping(monitor='loss')
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

checkpoint = ModelCheckpoint("model1.h5", monitor='val_loss', save_best_only=True)

#,validation_data = [test_pre,masks_pre_test],shuffle=True,steps_per_epoch=3

history = model.fit(images_train,masks_train,epochs = 100,callbacks=[checkpoint],validation_data=[images_val,masks_val])

In [ ]:
import matplotlib.pyplot as plt

# Assuming 'history' is the object returned by model.fit()

# Plot training & validation loss values
plt.figure(figsize=(12, 6))  # Adjust the figure size if needed
plt.subplot(1, 2, 1)  # Creating a subplot for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper left')

# Plot training & validation accuracy values
plt.subplot(1, 2, 2)  # Creating a subplot for accuracy
plt.plot(history.history['accuracy'])  # Assuming 'accuracy' is the training accuracy
plt.plot(history.history['val_accuracy'])  # Assuming 'val_accuracy' is the validation accuracy
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.tight_layout()  # Adjust layout to prevent clipping
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model
  # Replace with the path to your saved model

# Load a new image for prediction
img_path = 'Sample/23428960_15.png'  # Replace with the path to your new image
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)

img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
img_array = img_array/255.0
# Perform prediction
predictions = best_model.predict(img_array)

# Assuming your model outputs segmentation masks, you might want to threshold the predictions
threshold = 0.5  # Adjust as needed
predicted_mask = (predictions>threshold).astype(np.uint8)

# Visualize the original image, predicted mask, and overlay
original_image = image.load_img(img_path)
original_image_array = image.img_to_array(original_image) / 255.0  # Normalize to [0, 1]

plt.figure(figsize=(12, 6))
  # Adjust the target size to match your model's input size


# Plot original image
plt.subplot(1, 3, 1)
plt.imshow(original_image_array)
plt.title('Original Image')

# Plot predicted mask
plt.subplot(1, 3, 2)
plt.imshow(predicted_mask[0, :, :, 0], cmap='gray')  # Assuming a single-channel mask
plt.title('Predicted Mask')

# Plot overlay
plt.subplot(1, 3, 3)
plt.imshow(original_image_array)
plt.imshow(predictions[0, :, :, 0], cmap='gray', alpha=0.5)  # Adjust the colormap as needed
plt.title('Overlay')

plt.show()
